# Template for scraping

First inspect elements on the website to figure out the div and class of the target, and foudn that it's under table

In [2]:
baseURL = "http://www.learncpp.com"
r = urllib.urlopen(baseURL).read()
# or opinionPage = requests.get(matchURL, headers = hdr).text
soup = BeautifulSoup(r)
soup.prettify
body = soup.findAll("table")
# ind: 3-6 contains what I what
art_links = [baseURL + dd.a["href"] for dd in body[3].findAll("td") if dd.a and dd.a.has_attr("href") and len(dd.a["href"]) > 1]

In [ ]:
# find by attribute
listItem = soup.findAll(attrs={'class':'side-box'})

# Get title and link of each article
goalURLs = ''
for li in listItem:
    link = li.find('a')
    if link.attrs['title'] == 'See all player statistics':
        goalURLs = link.attrs['href']
        break

In [8]:
all_text = []
for link in art_links[8:17]:
    r = urllib.urlopen(link).read()
    soup = BeautifulSoup(r)
    head = soup.find("div", "post-headline")
    body = soup.find("div", "post-bodycopy")
    all_text = (all_text + 
                [head.find("h1").string.replace(u"\u2014", "--")] + 
                [' '.join(ii.strings).replace(u"\u2019", "'") for ii in body.findAll(["p", "li"]) if ii.strings][:-6] + 
                ['\n\n'])

In [9]:
with open("Output1.txt", "w") as text_file:
    for item in all_text:
        text_file.write("%s\n\n" % item.encode('utf8'))


## Or

In [6]:
baseURL = "http://www.learncpp.com"
r = urllib.urlopen(baseURL).read()
soup = BeautifulSoup(r)
soup.prettify
body = soup.findAll("div", "blueBubble")

In [56]:
for ii in body:
    aa = ii.parent.parent('td')[2]
    b = aa.findAll('a')[0]
    #print aa
    #print b
    #print b['href']

IndexError: list index out of range

# Example for selenium: Download Youtube videos as mp3

### Get all links of the playlist of popular videos of Ali

In [1]:
import urllib
from bs4 import *

baseURL = "https://www.youtube.com/watch?v=7SPivf0ZQzg&list=PL9NCK9qO2xk_poYOTaya1si7m0wx9K6-S"
r = urllib.urlopen(baseURL).read()
soup = BeautifulSoup(r)
body = soup.find('ol', 'playlist-videos-list yt-uix-scroller')

video_url_list = ["https://www.youtube.com" + link.a['href'] for link in body.findAll('li', 'yt-uix-scroller-scroll-unit')]


### Use existing website to convert each linked video to mp3 and download

Firefox annoyingly open mp3 with iTunes by default and won't save to the disk. A get-around is when the first download is made, select save to disk and do that for all following files. Then just keep looping and all later files will be saved to disk automatically


For usage of selenium, refer to:
http://irwinkwan.com/2013/04/05/automating-the-web-with-selenium-complete-tasks-automatically-and-write-test-cases/

In [10]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
    
# Create a new instance of the Firefox driver
fp = webdriver.FirefoxProfile()
# NEED TO USE THE CORRECT MIME TYPE HERE TO INDICATE FIREFOX SAVE A CERTAIN TYPE OF FILE TO DISK DIRECTLY
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "audio/mpeg3, audio/x-mpeg-3, video/mpeg, video/x-mpeg")
fp.set_preference("browser.download.manager.focusWhenStarting", False);
fp.set_preference("browser.download.manager.showWhenStarting", False);
fp.set_preference("browser.download.manager.showAlertOnComplete", False);

"""
fp.set_preference("plugin.disable_full_page_plugin_for_types", "audio/mpeg3")
fp.set_preference("browser.helperApps.alwaysAsk.force", False);
fp.set_preference("browser.download.manager.alertOnEXEOpen", False);
fp.set_preference("browser.download.manager.closeWhenDone", False);
fp.set_preference("browser.download.manager.useWindow", False);
fp.set_preference("services.sync.prefs.sync.browser.download.manager.showWhenStarting", False);
fp.set_preference("pdfjs.disabled", True);
fp.set_preference("browser.download.folderList",2)
fp.set_preference("browser.download.alertOnEXEOpen", False);
"""
driver = webdriver.Firefox(firefox_profile=fp)

# return error if waiting too long
driver.set_page_load_timeout(10)

web_url = 'http://www.vidtomp3.com/'

In [11]:
for video_url in video_url_list[0:1]:

    # go to the home page, locate the element for filling up the vidio link
    driver.get(web_url) 
    inputElement = driver.find_element_by_name("url")
    
    # switch window
    #driver.switch_to_window(driver.window_handles[1])
    
    # clear default text if necessary
    #inputElement.clear();

    # type in the search
    inputElement.send_keys(video_url)

    # submit the form
    inputElement.submit()
    
    # wait for the website to convert the video
    time.sleep(4)
    
    try:
        # get all links on the website, findout the one with "Download" in text, and click it
        ListlinkerHref = driver.find_elements_by_xpath("//*[@href]")
        for ii in ListlinkerHref:
            if "Download" in ii.text:
                break
        ii.click()
        
        # in the new page, find the link to the mp3 and click
        aa = driver.find_elements_by_id('downloadmp3')
        aa[0].click()
    except:
        pass

In [ ]:
# open new tab
body = driver.find_element_by_tag_name("body")
body.send_keys(Keys.COMMAND + 't')